### HTML 데이터 파싱을 이용한 크롤링
- 네이버 실시간 검색어 순위
    - HTML 포멧의 문자열을 가져오는 URL 찾음
    - 요청 -> 응답(HTML 포멧(문자열)의 데이터를 저장)
    - 데이터 파싱
        - bs4 패키지의 BeautifulSoup 클래스를 이용해서 dom(Document) 객체를 생성
        - dom 객체에서 우리가 원하는 데이터를 css-selector를 이용해서 수집
        - 데이터 프레임으로 만들기

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#### 1. requests 이용하여 HTML 코드 문자열 데이터 가져오기

In [4]:
url = "https://www.naver.com/"
response = requests.get(url)
response

<Response [200]>

#### 2. BeautifulSoup을 이용하여 css-selector를 사용할 수 있는 객체로 파싱

In [6]:
# html로 파싱
dom = BeautifulSoup(response.content, "html.parser")

#### 3. css-selector를 이용하여 원하는 데이터 수집
- select : 여러개의 엘리먼트 객체들을 리스트로 가져옴
    - type : list
- select_one : 하나의 엘리먼트 객체를 가져옴
    - type : bs4.element.Tag

In [14]:
# Chrome에 있는 copy selector 사용해도 된다.
elements = dom.select(".ah_roll_area > .ah_l > .ah_item")

In [15]:
type(elements[0]), elements[0]

(bs4.element.Tag, <li class="ah_item">
 <a class="ah_a" data-clk="lve.keyword" href="#">
 <span class="ah_r">1</span>
 <span class="ah_k">스텝업 입시부적</span>
 </a>
 </li>)

In [17]:
elements[0].select_one(".ah_k").text

'스텝업 입시부적'

#### 4. 수집한 데이터를 데이터 프레임으로 만들기

In [19]:
datas = []

for element in elements:
    # rank = element.select_one(".ah_r").text
    # keyword = element.select_one(".ah_k").text
    datas.append({
        "rank": element.select_one(".ah_r").text,
        "keyword": element.select_one(".ah_k").text,
    })

naver_keywords = pd.DataFrame(datas)
naver_keywords.tail(3)

,keyword,rank
17,김경수,18
18,수능 국어,19
19,목요일프렌즈타임,20


#### 5. 함수로 만들기

In [24]:
def naver_keywords():
    response = requests.get("https://www.naver.com/")
    dom = BeautifulSoup(response.content, "html.parser")
    elements = dom.select(".ah_roll_area > .ah_l > .ah_item")
    
    datas = []
    
    for element in elements:
        datas.append({
            "rank": element.select_one(".ah_r").text,
            "keyword": element.select_one(".ah_k").text,
        })
    
    return pd.DataFrame(datas)

In [25]:
df = naver_keywords()
df.tail(2)

,keyword,rank
18,소녀시대 유리,19
19,박은혜 남편,20


- 다음 실시간 검색어 순위 데이터 수집

In [90]:
response = requests.get("https://daum.net/")

In [91]:
dom = BeautifulSoup(response.content, "html.parser")

In [92]:
elements = dom.select(".realtime_part > .list_hotissue.issue_row > li")

In [93]:
len(elements)

10

In [94]:
rank = elements[0].select_one(".ir_wa").text
keyword = elements[0].select_one(".link_issue").text
print(rank, keyword)

1위 수능 시간표


In [95]:
datas = []
for element in elements:
    datas.append({
        "rank": element.select_one(".ir_wa").text,
        "keyword": element.select_one(".link_issue").text
    })
    
daum_keywords = pd.DataFrame(datas)
daum_keywords.tail(2)

,keyword,rank
8,설리,9위
9,수능 수학,10위


In [97]:
naver_df = naver_keywords()
daum_df = pd.DataFrame(datas)

In [99]:
set(naver_df["keyword"]) & set(daum_df["keyword"])

{'구본영', '김경수', '수능 수학', '수능 시간표', '수능 영어', '유리 오빠', '이문정'}